## 8.1 Implement a Stack with Max API

Design a stach taht includes a max operation, in addition to push and pop. The max method should return the maximum value stored in the stack. Hint: Use additional storage to track the maximum value. 

**Sol:** Suppose we use a single auxiliary variable, M, to record the element that is maximum in the stack. Updating M on pushes is easy: M= max(M,e), where e is the element being pushed. 

However, updating M on opo is very time consuming. If M is the element being popped, we have no way of knowing what the maximum remaining element is, and are forced to consider all the remining elements. 

We can dramatically improve on the time complexity of poping by caching, in essence, trading time for space. Specifically, for each entry in the stack, we cache the maximum stored at or below that entry. Now when we pop, we evict the corresponding cached value. 

In [2]:
import collections

In [3]:
class Stack:
    ElementWithCachedMax = collections.namedtuple('ElementWithCachedMax', 
                                                  ('element', 'max'))
    
    
    def __init__(self) -> None:
        self._element_with_cached_max: List[Stack.ElementWithCachedMax] = []
    
    def empty(self) -> bool:
        return len(self._element_with_cached_max) == 0
    
    def max(self) -> int:
        return self._element_with_cached_max[-1].max
    
    def pop(self) -> int:
        return self._element_with_cached_max.pop().element
    
    def push(self, x: int) -> None:
        self._element_with_cached_max.append(
         self.ElementWithCachedMax(x, x if self.empty() else max(x, self.max())))

Each of the specified methods has time complexity O(1). The additional space complexity is O(n) regardless of the stored keys. 

In [25]:
A = Stack()

In [26]:
A.push(1)

In [27]:
A.push(5)
A.push(0)
A.push(1)
A.push(2)
A.push(2)

In [8]:
A.pop()

2

In [10]:
A.max()

5

In [11]:
A.pop()

2

In [12]:
A.max()

5

In [29]:
def print_stack(A: Stack) -> None:
    while A.empty() is False:
        print(A.pop())

In [30]:
print_stack(A)

2
2
1
0
5
1


## 8.2 Evaluate RPN Expressions 

A string is said to be an arithmetical expression in Reverse Polish notation (RPN) if: 

* It is a single digit or a sequence of digits, prefixed with an option -, e.g.m "6", "123", "-42". 
* It is of the form "A, B, o" where A, B are RPN expressions and o is one of "+,-,$\times$, /"

An RPN expression can be evaluated uniquely to an integer, which is determined recursively. The base case corresponds to Rule(1.), which is an integer expressed in base-10 positional system. Rule(2.) on the preceding pagecorresponds to the recursive case, and the RPNs are evaluated in the natural way.

Write a program that takes an arithmetical expression in RPN and returns the number that the expression evaluates to. 

In [46]:
def evaluate(expression: str) -> int:
    intermediate_results: List[int] = []
    delimiter = ','
    operator = {
        '+': lambda y, x: x + y, '-': lambda y, x: x - y,
        '*': lambda y, x: x*y , '/': lambda y, x: x // y
    }
    
    for token in expression.split(delimiter):
        if token in operator:
            intermediate_results.append(operator[token](
                intermediate_results.pop(), intermediate_results.pop()))
        else: # token is a number 
            intermediate_results.append(int(token))
    return intermediate_results[-1]

In [40]:
a = '3,4,+,2,*,1,+'

In [43]:
delimiter = ','
for i in a.split(delimiter):
    print(i)

3
4
+
2
*
1
+


In [47]:
evaluate(a)

15

In [52]:
operator = {
    '+': lambda y, x: x + y, '-': lambda y, x: x - y,
    '*': lambda y, x: x * y, '/': lambda y, x: x /y
}

In [53]:
operator['+'](5,3)

8

In [54]:
operator['/'](5,3)

0.6

Since we perform O(1) computation per character of the string, the time complexity is O(n), where n is the length of the string. 

**Variant:** Solve the same problem for expressions in Polish notation, i.e. when A, B, o is replaced by o, A, B in Rule (2.) on the facing page. 

## 8.3 Is a String Well-formed?

A string over the characters "{,},[,],(,)" is said to be well-formed if the different types of brackets match in the correct order. 

Write a program that tests if a string made up of the characters '(',')','[',']','{','}'is well-formed. 

In [57]:
def is_well_formed(s: str) -> bool:
    left_chars, lookup = [], {'(': ')', '{': '}', '[':']'}
    for c in s:
        if c in lookup:
            left_chars.append(c)
        elif not left_chars or lookup[left_chars.pop()] != c:
            # Unmatched right char or mismatched chars
            return False
    return not left_chars

In [58]:
s = '{[]][[]]}'
is_well_formed(s)

False

In [59]:
s= '({[][]}())'
is_well_formed(s)

True

The time complexity is O(n) since for each character we perform O(1) operations. 

## 8.4 Normalize Pathnames 

A file or directory can be specified via a string called pathname. This string may specify an absolute path, starting from the root, e.g.,/usr/bin/gcc, or a path relative to the current working directory, e.g., scripts/awkscripts.

Write a program which takes a pathname, and returns the shortest equivalent pathname. Assume individual directories and files have names that use only alphanumeric characters. Subdirectory names may be combined using forward slashes (/), the current directory (.) and parent directory (..). 

In [61]:
def shortest_equivalent_path(path: str) -> str:
    if not path:
        raise ValueError('Empty string is not a valid path.')
    
    path_names = [] # Use list as a stack.
    # Special case: starts with '/', which is an aboslute path.
    if path[0] == '/':
        path_names.append('/')
    
    for token in (token for token in path.split('/')
                  if token not in ['.', '']):
        if token == '..':
            if not path_names or path_names == '..':
                path_names.append(token)
            else:
                if path_names[-1] == '/':
                    raise ValueError('Path Error')
                path_names.pop()
        else: # Must be a name
            path_names.append(token)
    
    
    result = '/'.join(path_names)
    return result[result.startswith('//'):] # Avoid starting with '//'

In [63]:
path = 'sc//./../tc/awk/././'
shortest_equivalent_path(path)

'tc/awk'

In [64]:
path = '/usr/bin/../lib/gcc'
shortest_equivalent_path(path)

'/usr/lib/gcc'

In [66]:
path = 'scripts//./../scripts/awkscrits/././'
shortest_equivalent_path(path)

'scripts/awkscrits'

The time complexity is O(n), where n is the length of the pathname. 

## 8.5 Compute Building with a Sunset View 

You are given a series of buildings that have windows facing west. The buildings are in a straight line, and any building which is to the east of a building of equal or greater height cannot view the sunset. 

Design an algorithm that processes buildings in east-to-west order and returns the set of buildings which view the sunset. Each buildings is specified by its height. 

**Sol:** A brute-force approach is to store all buildings in an array. We then do a reverse scan of this array, tracking the running maximum. Any building whose height is less than or equal to the running maximum does not have a sunset view. 

The time and space complexity are both O(n), where n is the number of buildings. 

If a new buidlings is shorter than a building in the current set, then all buildings in the current set which are further to the east cannot be blocked by the new building. This suggests keeping the buildings in a last-in, first-out manner, so that we can terminate earlier. 

Specifically, we use a stack to record buildings that have a view. Each time a building b is processed, if it is taller than the building at the top of the stack, wepop the stack unitl the top of the stack is taller than b--all the buildings thus removed lie to the east of a taller building. 

Althouhg some individual steps may require many pops, each building is pushed and popped at most once. Therefore, the run time to process n buildings is O(n), and the stack always holds precisely the buildings which currently have a view. 

The memory used is O(n), and the bound is tight, even when only one building has a view--consider the input where the west-most building is the tallest, and the remaining n-1 buildings decrease in height from east to west. However, in the best-case, e.g., when buildings appear in increasing height, we use O(1) space. In contrast, the brute-force approach always uses O(n) space. 

In [68]:
def examine_buildings_with_sunset(sequence) -> list:
    BuildingWithHeight = collections.namedtuple('BuildingWithHeight',
                                               ('id', 'height'))
    candidates: List[BuildingWithHeight] = []
    for building_idx, building_height in enumerate(sequence):
        while candidates and building_height >= candidates[-1].height:
            candidates.pop()
        candidates.append(BuildingWithHeight(building_idx, building_height))
    return [c.id for c in reversed(candidates)]